
# Convolutional Neural Networks for Image Classification

In [2]:
import pandas as pd
import numpy as np

In [1]:
from tensorflow.keras.datasets import mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()

##  Visualizing the Image Data

In [3]:
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
x_train.shape

In [19]:
single_image = x_train[6]

In [20]:
single_image

In [21]:
single_image.shape

In [22]:
plt.imshow(single_image)

# PreProcessing Data

We first need to make sure the labels will be understandable by our CNN.

## Labels

In [13]:
y_train.shape

In [8]:
y_test

Looks like our labels are literally categories of numbers. We need to translate this to be "one hot encoded" so our CNN can understand, otherwise it will think this is some sort of regression problem on a continuous axis. Luckily , Keras has an easy to use function for this:

In [9]:
from tensorflow.keras.utils import to_categorical

In [10]:
y_train.shape

In [11]:
y_example = to_categorical(y_train)

In [23]:
y_example[6]

In [15]:
y_example.shape

In [16]:
y_example[0]

In [17]:
y_cat_test = to_categorical(y_test,10)

In [18]:
y_cat_train = to_categorical(y_train,10)

### Processing X Data

We should normalize the X data

In [24]:
single_image.max()

In [25]:
single_image.min()

In [26]:
x_train = x_train/255
x_test = x_test/255

In [27]:
scaled_single = x_train[0]

In [28]:
scaled_single.max()

In [26]:
plt.imshow(scaled_single)

## Reshaping the Data

Right now our data is 60,000 images stored in 28 by 28 pixel array formation. 

This is correct for a CNN, but we need to add one more dimension to show we're dealing with 1 RGB channel (since technically the images are in black and white, only showing values from 0-255 on a single channel), an color image would have 3 dimensions.

In [29]:
x_train.shape

In [30]:
x_test.shape

Reshape to include channel dimension (in this case, 1 channel)

In [31]:
x_train = x_train.reshape(60000, 28, 28, 1)

In [32]:
x_train.shape

In [33]:
x_test = x_test.reshape(10000,28,28,1)

In [34]:
x_test.shape

# Training the Model

In [36]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten

In [37]:
model = Sequential()

# CONVOLUTIONAL LAYER
model.add(Conv2D(filters=32, kernel_size=(4,4),input_shape=(28, 28, 1), activation='relu',))
# POOLING LAYER
model.add(MaxPool2D(pool_size=(2, 2)))

# FLATTEN IMAGES FROM 28 by 28 to 764 BEFORE FINAL LAYER
model.add(Flatten())

# 128 NEURONS IN DENSE HIDDEN LAYER (YOU CAN CHANGE THIS NUMBER OF NEURONS)
model.add(Dense(128, activation='relu'))

# LAST LAYER IS THE CLASSIFIER, THUS 10 POSSIBLE CLASSES
model.add(Dense(10, activation='softmax'))

# https://keras.io/metrics/
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy']) # we can add in additional metrics https://keras.io/metrics/

In [38]:
model.summary()

In [39]:
from tensorflow.keras.callbacks import EarlyStopping

In [40]:
early_stop = EarlyStopping(monitor='val_loss',patience=2)

## Train the Model

In [41]:
model.fit(x_train,y_cat_train,epochs=10,validation_data=(x_test,y_cat_test),callbacks=[early_stop])

## Evaluate the Model

In [42]:
model.metrics_names

In [43]:
losses = pd.DataFrame(model.history.history)

In [44]:
losses.head()

In [45]:
losses[['accuracy','val_accuracy']].plot()

In [46]:
losses[['loss','val_loss']].plot()

In [47]:
print(model.metrics_names)
print(model.evaluate(x_test,y_cat_test,verbose=0))

In [48]:
from sklearn.metrics import classification_report,confusion_matrix

In [93]:
predictions = model.predict(x_test)

In [51]:
y_cat_test.shape

In [52]:
y_cat_test[0]

In [61]:
predictions[0].round()

In [62]:
y_test

In [113]:
L = [0, 0, 1, 0]
L.index(1)

In [111]:
predictions = model.predict(x_test)
predictions = [np.where(p.round() == 1.0) for p in predictions]
#predictions
print(classification_report(y_test,predictions))
#y_test[3] == predictions[3]

In [60]:
confusion_matrix(y_test,predictions)

In [59]:
import seaborn as sns

In [104]:
plt.figure(figsize=(10,6))
sns.heatmap(confusion_matrix(y_test,predictions),annot=True)
# https://github.com/matplotlib/matplotlib/issues/14751

# Predicting a given image

In [84]:
my_number = x_test[89]

In [85]:
plt.imshow(my_number.reshape(28,28))

In [86]:
# SHAPE --> (num_images,width,height,color_channels)
model.predict(my_number.reshape(1,28,28,1)).round()

Looks like the CNN performed quite well!